In [1]:
import pandas as pd

In [2]:
from google.colab import files
uploaded = files.upload()

Saving test.csv to test.csv


In [3]:

data = pd.read_csv('test.csv',encoding='latin1')

In [4]:
data

,textID,text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral,morning,0-20,Afghanistan,38928346.0,652860.0,60.0
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive,noon,21-30,Albania,2877797.0,27400.0,105.0
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative,night,31-45,Algeria,43851044.0,2381740.0,18.0
3,01082688c6,happy bday!,positive,morning,46-60,Andorra,77265.0,470.0,164.0
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive,noon,60-70,Angola,32866272.0,1246700.0,26.0
...,...,...,...,...,...,...,...,...,...
4810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4811,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4813,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
data.columns

Index(['textID', 'text', 'sentiment', 'Time of Tweet', 'Age of User',
       'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)'],
      dtype='object')

In [6]:
data = data[['text','sentiment']]

In [7]:
data

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely -...,positive
2,"Recession hit Veronique Branquinho, she has to...",negative
3,happy bday!,positive
4,http://twitpic.com/4w75p - I like it!!,positive
...,...,...
4810,NaN,NaN
4811,NaN,NaN
4812,NaN,NaN
4813,NaN,NaN


In [8]:
data = data.dropna()

In [9]:
import numpy as np

In [10]:
data.isnull().sum()

,0
text,0
sentiment,0


In [21]:
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text
tokens = tokenizer(
    list(data['text']),
    max_length=128,
    truncation=True,
    padding='max_length',
    return_tensors='tf'
)

# Encode sentiment labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['sentiment'])


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.


In [30]:
data

,text,sentiment
0,Last session of the day http://twitpic.com/67ezh,1
1,Shanghai is also really exciting (precisely -...,2
2,"Recession hit Veronique Branquinho, she has to...",0
3,happy bday!,2
4,http://twitpic.com/4w75p - I like it!!,2
...,...,...
3529,"its at 3 am, im very tired but i can`t sleep ...",0
3530,All alone in this old house again. Thanks for...,2
3531,I know what you mean. My little dog is sinkin...,0
3532,_sutra what is your next youtube video gonna b...,2


In [23]:
import tensorflow as tf

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'input_ids': tokens['input_ids'],
        'attention_mask': tokens['attention_mask']
    },
    labels
))

# Shuffle and batch
BATCH_SIZE = 32
dataset = dataset.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [24]:
from transformers import TFBertForSequenceClassification

# Number of classes
num_classes = len(label_encoder.classes_)

# Load pre-trained BERT for classification
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)


TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)


In [26]:
history = model.fit(dataset, epochs=3)

Epoch 1/3
111/111 [==============================] - 129s 748ms/step - loss: 0.8686 - accuracy: 0.5897
Epoch 2/3
111/111 [==============================] - 87s 787ms/step - loss: 0.5562 - accuracy: 0.7770
Epoch 3/3
111/111 [==============================] - 87s 785ms/step - loss: 0.3963 - accuracy: 0.8588


In [28]:
def predict_sentiment(text):
    # Tokenize the input text
    tokens = tokenizer(
        [text],  # Pass as a list since the tokenizer expects a list of strings
        max_length=128,
        truncation=True,
        padding='max_length',
        return_tensors='tf'
    )

    # Make prediction
    predictions = model.predict({
        'input_ids': tokens['input_ids'],
        'attention_mask': tokens['attention_mask']
    })

    # Get the predicted class index
    predicted_class_index = tf.argmax(predictions.logits, axis=1).numpy()[0]

    # Map the index back to the original sentiment label
    predicted_sentiment = label_encoder.inverse_transform([predicted_class_index])[0]

    return predicted_sentiment

# Example usage:
# text_to_predict = "This is a great day!"
# sentiment = predict_sentiment(text_to_predict)
# print(f"The sentiment of the text is: {sentiment}")

In [ ]:
# Example usage:
text_to_predict = input("Enter the text you want to analyze: ")
sentiment = predict_sentiment(text_to_predict)
print(f"The sentiment of the text is: {sentiment}")